#Coding Week ML TASK1
 
This notebook includes all steps including EDA,cleaning,preprocessing, model training and evaluation that lies behind the solution of provided ML task. 

In [ ]:
#first import all the necessary libraries
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
#for scikit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [ ]:
#firstly we load dataset using pandas
#let's name dataset df for simplicity
df = pd.read_csv("tour_logs_test_input.csv")


Now that the dataset is loaded, inspect few first rows using head(); we can even load few rows from back using tail()

In [ ]:
#first few rows
df.head()
#last few rows
df.tail()
#we can also load a few rowsncols using loc function
df.loc[28:39]

#now we can load only certain specific cols 
#for better exploration of data before analysisng
df.loc[31:69,['Venue_ID','Ticket_Price','Crowd_Size']]

Now we check basics for given dataset.

In [ ]:
#information about df
df.info()
#summary of df
df.describe()
#size of df
print(df.size())
#shape of df ie rows by cols
print(df.shape)
#for columns
print(df.columns)


Data preprocessing:
handling missing values, feature scaling, etc

We can firstly drop col that are unnecessary in EDA such as gig_id

In [ ]:
df1 = df.copy()
df1.drop(column=['Gig_ID'],inplace=True)

Then we find null values in remaning rows

In [ ]:
#finding null values
#using this we find how many cols have null values
print(df1.isnull().sum())
#percentage of null rows
print(df1.isnull().sum() * 100)


Here  volume_level and crowd_size have 8.4% and 2.2% missing values only; which is comparitively quiet small compared to number of rows present in dataset.
so, we can drop them

In [ ]:
#dropping null values
df1.dropna(inplace=True)

Identifying and handling duplicate values


In [ ]:
#identifying rows which are exact duplicates
duplicate_rows = df1.duplicated()
print('duplicated rows:', duplicate_rows.sum())
#removing them
df1 = df1.drop_duplicates()
#drop_duplicates() removes exact row duplicates keeping first common occurance


Fixing errors in dataset
-The dates are inconsistent but in data evaluation those won't come handy as we have already dropped earlier
-Ticket prices are inconsistent and are to be converted in a certain single unit.
-applying certain changes like absolute and constraints 

We have prices in other units. so, we run another function to convert them all

In [ ]:
#converting
def conversion_in_usd(y):
    y = str(y).strip()
    
    if y.lower() == 'free':
        return 0.0  # free ticket
    
    # take first meaningful part
    a = y.split()[0]

    if a.startswith('£'):
        return float(a[1:]) * 1.27
    elif a.startswith('€'):
        return float(a[1:]) * 1.09
    elif a.startswith('$'):
        return float(a[1:])
    elif a.endswith('USD'):
        return float(a.replace('USD', ''))
    else:
        return float(a)
df1['Ticket_Price_USD'] = df1['Ticket_Price'].apply(conversion_in_usd)
print(df1['Ticket_Price_USD'])


As the date & time are in inconsistent format, we run a function to make them in one consistent format YYYY-MM-DD and HH:MM:SS 

First we will work on date

We'll handle the textual ones first

In [ ]:
df1.loc[df1['Show_Date'].str.contains('morning', case=False, na=False), 'Time_Bucket'] = 'Morning'
df1.loc[df1['Show_Date'].str.contains('afternoon', case=False, na=False), 'Time_Bucket'] = 'Afternoon'
df1.loc[df1['Show_Date'].str.contains('evening', case=False, na=False), 'Time_Bucket'] = 'Evening'
df1.loc[df1['Show_Date'].str.contains('late night', case=False, na=False), 'Time_Bucket'] = 'Late Night'



for dd/mm/yyyy

In [ ]:
ddmmyyyy = df1['Show_Date'].str.contains(r'\d{2}/\d{2}/\d{4}', na=False)
df1.loc[ddmmyyyy, 'Clean_Date'] = pd.to_datetime(
    df1.loc[ddmmyyyy, 'Show_Date'],
    dayfirst=True,
    errors='coerce'
)


for yyyy-dd-mm

In [ ]:
mask_yyyyddmm = df1['Show_Date'].str.contains(r'\d{4}-\d{2}-\d{2}', na=False)
df1.loc[mask_yyyyddmm, 'Clean_Date'] = pd.to_datetime(
    df1.loc[mask_yyyyddmm, 'Show_Date'],
    format='%Y-%d-%m',
    errors='coerce'
)


for (month date, year) type

In [ ]:
#renaming for ease in code
df1['Show_DateTime']=df1['datentime']
#removing uncessary spaces
df1['datentime']=df1['datentime'].str.strip()
#writing function
def parse_date(x):
    try:
        # If full MonthDayYear
        return pd.to_datetime(x, format='%b%d,%Y')
    except:
        try:
            # If only Day,Year → add default month (e.g., Jan)
            return pd.to_datetime('Jan' + x, format='%b%d,%Y')
        except:
            try:
                # If only MonthDay → add default year (e.g., 2024)
                return pd.to_datetime(x + ',2024', format='%b%d,%Y')
            except:
                return pd.NaT

df1['Clean_Date'] = df1['Show_Date'].apply(parse_date)
print(df1)


some rows include volume rate as negatives; so we replace such values in positives as it's likely a data entry error

In [ ]:
df1['Volume_Level']=df1['Volume_Level'].abs()
print(df1['Volume_Level'])

Now that we have clean dataset; we can move forward to EDA

EDA:
-understanding patterns
we can visualise parameters available like day_of_week over exploring possibility of ticket_price

Checking venue categorisation

In [ ]:
#count
print(df1['Venue_ID'].value_counts())
print(df1.groupby('Venue_ID')['Crowd_Size'].std())
#plot
sns.countplot(x='Venue_ID', data=df1)
plt.title('Number of Shows per Venue')
plt.show()


TESTING AS PER SCRIBBLES
-tracking for day of week ~ (tuesdays curse?)

 checking for other factors like volume level, weather, etc if they roleplay in affecting
 tuesday shows

In [ ]:
#comparing tuesdays with other days
#mean
print(df1.groupby('Day_of_Week')['Crowd_Size'].mean())
#variance
print(df1.groupby('Day_of_Week')['Volume_Level'].std())
#plot
sns.boxplot(x='Day_of_Week', y='Opener_Rating', data=df1)
plt.show()
#mean
print(df1.groupby('Day_of_Week')['Opener_Rating'].mean())
#for tuesdays only
tuesdays = df1[df1['Day_of_Week'] == 1]
print("Tuesday shows statistics:")
print(tuesdays.describe())
#plot
sns.boxplot(x='Day_of_Week', y='Crowd_Size', data=df1)
plt.title('Crowd Size by Day of Week')
plt.show()


Here tuesdays have 498.85 as mean which is less compared to comparitive average of 533.71 over week; On tuedays, we have volume level's mean = 5.48 which is comparable to avg value of 523 Here, tuesday's have variance of 3.51 which is more compared to avg of 3.33; but it doesn't have highest std so it aint that unstable

Calculating bad days as those when volume level is greater than average levels; then compare it for days in week.

In [ ]:
df1['Bad_Event'] = (df1['Volume_Level'] < df1['Volume_Level'].mean()).astype(int)
print(df1.groupby('Day_of_Week')['Bad_Event'].mean())

On tuesdays, we get 0.56 around which is slight higher than avg value 0.54

-Next, 
-full moon=best shows
-volume level
-leather jacket-mid
-spendex right- but data otherwise
-rain sucks
-v delta analysis
-mosh pits

moon-phase and show analysis

In [ ]:
#moon phase x vol levels
print(df1.groupby('Moon_Phase')['Volume_Level'].mean())
#moon phase x crowd size
print(df1.groupby('Moon_Phase')['Crowd_Size'].mean())
#mean
print(df1.groupby('Moon_Phase')['Opener_Rating'].mean())
#count
df1.groupby('Moon_Phase')['Opener_Rating'].count()
#plot moon phase x vol level
sns.boxplot(x='Moon_Phase', y='Volume_Level', data=df1)
plt.show()
#plot moon phase x crowd size
sns.boxplot(x='Moon_Phase', y='Crowd_Size', data=df)
plt.show()

full moon = 5.14; avg is 5.9 is very small-
it's quiet less compared to avg
least on full moon - 491.07
From boxplot, we observe full moon has 4.92 outlier with mean 5.14 and nothing exceptional

Outfit via checking

In [ ]:
#band fit x crowd size
print(df1.groupby('Band_Outfit')['Crowd_Size'].mean())

Leather jacket has 512.38 mean crowd size compared to avg of 537.87; which quiet less compared to usual.
outfit analysis - spandex feels right n has 541.53 avg;
velvet-546.62 n highest is denim-550.95

weather wise analysis

In [ ]:
print(df1.groupby('Weather')['Crowd_Size'].mean())
print(df1.groupby('Weather')['Opener_Rating'].mean())
#venue related
print(df1.groupby(['Venue_ID','Weather'])['Opener_Rating'].mean())
#plot
sns.boxplot(x='Weather', y='Crowd_Size', data=df1)
plt.show()


obeserved from boxplot that cloudy-508, clear-529,stormy-512, rainy-532
mean- full moon=491.07 less-least

mosh pits analysis - 
-monks vs mosh pit

-analysed for: volume levels, variance level, crowd size

In [ ]:
#venue wise analysing
df.groupby('Venue_ID')['Volume_Level'].describe()
#mosh pits at v_delta
delta = df1[df1['Venue'] == 'V_Delta']
sns.scatterplot(x='Volume_Level', y='Crowd_Size', data=delta)
plt.show()


-pricing sensitivity across levels based on ticket_prices
-reactions-ratings as per ticket prices groupwise analysed

Creating a price bucket using certain range to test singer's claims 

In [ ]:
#fixing certain range
def price_bucket(p):
    if p < 50:
        return 'Low (<50)'
    elif 50 <= p < 60:
        return 'Mid (50 to 60)'
    elif 60 <= p < 70:
        return 'Sweet Spot (60 to 70)'
    else:
        return 'High (70+)'
#
df1['Price_Bucket'] = df1['Ticket_Price_USD'].apply(price_bucket)
print(df1['Price_Bucket'])


price x ratings

In [ ]:
# Barplot for average rating by price bucket
sns.barplot(x='Price_Bucket', y='Opener_Rating', data=df1, ci=None, palette='viridis')
plt.title('Average Rating by Ticket Price Bucket')
plt.ylabel('Average Rating (0-5)')
plt.xlabel('Price Bucket')
plt.xticks(rotation=15)  # rotate if needed
plt.show()

price x rating x venue


In [ ]:
#correlating by vol level
print(df1[['Volume_Level','Crowd_Size','Opener_Rating']].corr())
#corr by venue
print(df1.groupby('Venue_ID')[['Volume_Level','Crowd_Size','Opener_Rating']].corr())
price_order = ['Low (<50)', 'Mid (50–60)', 'Sweet Spot (60–70)', 'High (70+)']
sns.barplot(
    x='Price_Bucket',
    y='Opener_Rating',
    hue='Venue_ID',
    data=df1,
    ci=None,
    order=price_order
)
plt.title('Ratings Across Price Buckets per Venue')
plt.ylabel('Average Rating (0-5)')
plt.xlabel('Price Bucket')
plt.legend(title='Venue')
plt.show()

EDA was done on tour_logs_test_inputs.csv 
now for feature engineering and modeling use tour_logs_train.csv

making certain important data curation on train.csv too cause the dataset is unfiltered and messy

In [ ]:
df2 = pd.read_csv("tour_logs_train.csv")

Feature engineering
-Droping leakage cols
-extracting important features

Doing all basics for train.csv too

In [ ]:
print(df2)
#information about df
df2.info()
#summary of df
df2.describe()
#size of df
print(df2.size)
#shape of df ie rows by cols
print(df2.shape)
#for columns
print(df2.columns)
print(df2.isnull().sum())
#percentage of null rows
print(df2.isnull().sum() * 100)
df2.dropna(inplace=True)
#identifying rows which are exact duplicates
duplicate_rows = df2.duplicated()
#removing 
#prices in usd
def conversion_in_usd(y):
    y = str(y).strip()
    
    if y.lower() == 'free':
        return 0.0  # free ticket
    
    # take first meaningful part
    a = y.split()[0]

    if a.startswith('£'):
        return float(a[1:]) * 1.27
    elif a.startswith('€'):
        return float(a[1:]) * 1.09
    elif a.startswith('$'):
        return float(a[1:])
    elif a.endswith('USD'):
        return float(a.replace('USD', ''))
    else:
        return float(a)

df2['Ticket_Price_USD'] = df2['Ticket_Price'].apply(conversion_in_usd)
#dates
df2.loc[df2['Show_DateTime'].str.contains('morning', case=False, na=False), 'Time_Bucket'] = 'Morning'
df2.loc[df2['Show_DateTime'].str.contains('afternoon', case=False, na=False), 'Time_Bucket'] = 'Afternoon'
df2.loc[df2['Show_DateTime'].str.contains('evening', case=False, na=False), 'Time_Bucket'] = 'Evening'
df2.loc[df2['Show_DateTime'].str.contains('late night', case=False, na=False), 'Time_Bucket'] = 'Late Night'
#dd-mm-yyyy
ddmmyyyy = df2['Show_DateTime'].str.contains(r'\d{2}/\d{2}/\d{4}', na=False)
df2.loc[ddmmyyyy, 'Clean_Date'] = pd.to_datetime(
    df2.loc[ddmmyyyy, 'Show_DateTime'],
    dayfirst=True,
    errors='coerce'
)
#yyyy-dd-mm
mask_yyyyddmm = df2['Show_DateTime'].str.contains(r'\d{4}-\d{2}-\d{2}', na=False)
df2.loc[mask_yyyyddmm, 'Clean_Date'] = pd.to_datetime(
    df2.loc[mask_yyyyddmm, 'Show_DateTime'],
    format='%Y-%d-%m',
    errors='coerce'
)
#renaming for ease in code
#removing uncessary spaces
df2['Show_DateTime']=df2['Show_DateTime'].str.strip()
#writing function
def parse_date(x):
    try:
        # If full MonthDayYear
        return pd.to_datetime(x, format='%b%d,%Y')
    except:
        try:
            # If only Day,Year → add default month (e.g., Jan)
            return pd.to_datetime('Jan' + x, format='%b%d,%Y')
        except:
            try:
                # If only MonthDay → add default year (e.g., 2024)
                return pd.to_datetime(x + ',2024', format='%b%d,%Y')
            except:
                return pd.NaT
df2['Clean_Date'] = df2['Show_DateTime'].apply(parse_date)
#correcting negative volume level
df2['Volume_Level']=df2['Volume_Level'].abs()
#again dropping null values if any present
df2.dropna(inplace=True)

Droping merch sale with certain cols is significant because this data is recorded post-show and ain't involved in prediction time

In [ ]:
leakage_cols = [['Merch_Sales_Post_Show'],['Moon_Phase'],['Band_Outfit']]
df2 = df2.drop(columns=leakage_cols)
df2.shape
print(df2.info())


Extracting features 

In [ ]:
#price per capacity ratio
df2['Price_per_Crowd'] = df2['Ticket_Price_USD'] / (df2['Crowd_Size'] + 1)
#crowd density
#using manager's venue capacity
df2['Crowd_Density'] = df2['Crowd_Size'] / 800


In [ ]:
#weekend wise
df2['Is_Weekend'] = df2['Day_of_Week'].isin(
    ['Friday', 'Saturday']
).astype(int)

#making time bucket
def time_bucket(hour):
    if hour < 12:
        return 'Morning'
    elif hour < 17:
        return 'Afternoon'
    elif hour < 21:
        return 'Evening'
    else:
        return 'Late Night'

df2['Time_Bucket'] = df2['Show_DateTime'].dt.hour.apply(time_bucket)

Venue wise 

In [ ]:
#count x venue
df2['Show_Count_Venue'] = df2.groupby('Venue_ID').cumcount()
#vol x venue
df2['Volume_x_Venue'] = df2['Volume_Level'] * df2['Venue_ID'].astype('category').cat.codes
#price x venue
df2['Price_x_Venue'] = df2['Ticket_Price'] * df2['Venue_ID'].astype('category').cat.codes
#weather x venue
df2['Bad_Weather'] = df2['Weather'].isin(['Rain', 'Storm']).astype(int)
df2['Weather_x_Venue'] = df2['Bad_Weather'] * df2['Venue_ID']


Log transfromed skew features

In [ ]:
df2['Log_Crowd_Size'] = np.log1p(df2['Crowd_Size'])
df2['Log_Ticket_Price'] = np.log1p(df2['Ticket_Price'])

checking Stablity with certain new indexes

In [ ]:
#helps in mosh pits feedback
df2['Chaos_Index'] = (
    df2['Volume_Level'] * df2['Crowd_Density']
)
#pricing based
#helps in understanding here relative > absolute pricing
df2['Price_Shock'] = (
    df2['Ticket_Price_USD'] > df2.groupby('Venue_ID')['Ticket_Price_USD'].transform('mean')
).astype(int)


In [ ]:
#new dataset concisely having new features added
df_model = df2.copy()

Now, we head to modelling 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
y = pd.to_numeric(df2['Crowd_Energy'], errors='coerce')
X = df2.drop(columns=['Crowd_Energy'])
# Clean NaNs from Target
mask = y.notna()
y = y[mask]
X = X.loc[mask]
# Clean Features:
# Drop Date/Time columns
# Keep only Numbers and Text
X = X.select_dtypes(include=['number', 'object', 'category'])
# Encode Text to Numbers
X = pd.get_dummies(X, drop_first=True, dtype=int)
# Force float just to be safe
X = X.astype(float)
# Train-Test Split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)
#Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_preds = lr.predict(X_val)
# FIX: Use np.sqrt() instead of squared=False
lr_rmse = np.sqrt(mean_squared_error(y_val, lr_preds))
print(f"Linear Regression RMSE: {lr_rmse:.4f}")
#Decision Tree 
dt = DecisionTreeRegressor(random_state=42)
dt_param_grid = {
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10]
}

dt_grid = GridSearchCV(dt, dt_param_grid, cv=5, scoring='neg_root_mean_squared_error')
dt_grid.fit(X_train, y_train)

dt_preds = dt_grid.predict(X_val)
dt_rmse = np.sqrt(mean_squared_error(y_val, dt_preds)) # FIX
print(f"Decision Tree RMSE:     {dt_rmse:.4f}")

#Random Forest
rf = RandomForestRegressor(random_state=42)
rf_param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [10, 20]
}
rf_grid = GridSearchCV(rf, rf_param_grid, cv=5, scoring='neg_root_mean_squared_error')
rf_grid.fit(X_train, y_train)

rf_preds = rf_grid.predict(X_val)
rf_rmse = np.sqrt(mean_squared_error(y_val, rf_preds))
print(f"Random Forest RMSE:     {rf_rmse:.4f}")
#Comparison: Default Random Forest vs Tuned Random Forest
#Training a basic Random Forest with no hyperparameter tuning
rf_default = RandomForestRegressor(random_state=42)
rf_default.fit(X_train, y_train)
rf_default_preds = rf_default.predict(X_val)
rf_default_rmse = np.sqrt(mean_squared_error(y_val, rf_default_preds))

print(f"Random Forest (Default) RMSE: {rf_default_rmse:.4f}")
print(f"Random Forest (Tuned)   RMSE: {rf_rmse:.4f}")

# Improvement Calculation
improvement = rf_default_rmse - rf_rmse
print(f"Hyperparameter tuning improved RMSE by: {improvement:.4f}")

Random Forest RMSE:     51.8281
Random Forest (Default) RMSE: 51.8279
Random Forest (Tuned)   RMSE: 51.8281
Hyperparameter tuning improved RMSE by: -0.0002

Finally loading in predictions.csv

In [ ]:
test_df = pd.read_csv('sample_submission.csv')
#Encode categorical variables
X_test_submission = pd.get_dummies(test_df, drop_first=True, dtype=int)

#Aligning columns with Training Data
#adding missing columns (filled with 0) and removes extra cols
X_test_submission = X_test_submission.reindex(columns=X.columns, fill_value=0)

#Generate Predictions using your Best Model
from sklearn.ensemble import RandomForestRegressor
best_params = {
    'n_estimators': 100,
    'max_depth': 10,
    'min_samples_split': 2,
    'min_samples_leaf': 1,
    'random_state': 42
}
rf_best = RandomForestRegressor(**best_params)
rf_best.fit(X_train, y_train)
final_predictions = rf_best.predict(X_test_submission)
#Saving CSV 
submission = pd.DataFrame({
    'Gig_Id': test_df.index, 
    'Crowd_Energy': final_predictions
})
submission.to_csv('sample_submission.csv', index=False)
print("predictions.csv generated successfully!!!")

revenue optimisation

In [ ]:
# lists to store my results
prices = []
profits = []
audience = []
energies = []
# checking prices from 50 to 200 with gap of 5
for p in range(50, 201, 5):
    # linear formula for audience
    # if price is 0, audience is 800. if price is 200, audience is 0
    a = 800 * (1 - (p/200))
    a = int(a) # people can't be float
    
    if a < 0:
        a = 0
        
     #making a temporary row to predict energy
     #used the average values for other columns
    row = {
        'Venue_ID': 'V_Gamma',
        'Ticket_Price_USD': p,
        'Crowd_Size': a,
        'Volume_Level': 4.5, # average volume
        'Day_of_Week': 'Saturday', # assume weekend
        'Weather': 'Clear',
        'Show_DateTime': pd.to_datetime('2024-01-01 20:00:00')
        #certain date time taken to avoid crashing of code
    }
    
    #into dataframe
    temp_df = pd.DataFrame([row])
    #feature engineering for rows
    temp_df['Price_per_Crowd'] = temp_df['Ticket_Price_USD'] / (temp_df['Crowd_Size'] + 1)
    temp_df['Crowd_Density'] = temp_df['Crowd_Size'] / 800
    temp_df['Time_Bucket'] = 'Evening'
    temp_df['Is_Weekend'] = 1
    temp_df['Log_Crowd_Size'] = np.log1p(temp_df['Crowd_Size'])
    temp_df['Log_Ticket_Price'] = np.log1p(temp_df['Ticket_Price_USD'])
    temp_df['Chaos_Index'] = temp_df['Volume_Level'] * temp_df['Crowd_Density']
    
    #calculating price shock manually
    avg_price_gamma = df2[df2['Venue_ID'] == 'V_Gamma']['Ticket_Price_USD'].mean()
    if p > avg_price_gamma:
        temp_df['Price_Shock'] = 1
    else:
        temp_df['Price_Shock'] = 0
        
    #encoding
    temp_df = pd.get_dummies(temp_df, drop_first=True, dtype=int)
    
    #fixing missing columns
    #aligning with X_train columns from before
    temp_df = temp_df.reindex(columns=X_train.columns, fill_value=0)
    #predicting energy
    energy = rf_best.predict(temp_df)[0]
    #calculating money
    #ticket sales
    money_tickets = p * a
    
    #extra commodities(drinks/merch)
    #assuming base spend is 20 and energy increases it
    extra_spend = 20 * (1 + energy/200)
    money_extra = a * extra_spend
    #costs
    cost = 5000 + (8 * a) #8 per head(given)
    #total profit
    total = money_tickets + money_extra - cost
    #adding to list
    prices.append(p)
    profits.append(total)
    audience.append(a)
    energies.append(energy)

#finding the max profit
max_p = max(profits)
index_best = profits.index(max_p)
best_price = prices[index_best]
print("Best Price is:", best_price)
print("Max Profit is:", max_p)
print("Audience at best price:", audience[index_best])

Plotting curve

In [ ]:
#plotting the graph
plt.plot(prices, profits)
plt.title("Profit vs Price")
plt.xlabel("Price")
plt.ylabel("Profit")
plt.axvline(x=best_price, color='r', linestyle='--')
plt.show()

Thank you ☺️